In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import plotly.figure_factory as ff
import re

In [3]:
us_ig = pd.read_csv('../Dash_app/data/historical_ig.csv')

In [4]:
exclude_these = [25012986,
25012688,
25012758,
25012982,
25012981,
25012980,
25010801,
25012776,
25012798,
25012590,
25012443,
25012185,
25012125,
25011797,
25011968,
25012018,
25011842,
25011510,
25011637,
25011636,
25011635,
25011550,
25011416,
25011618,
25011319,
25011318,
25011317,
25011159,
25011158,
25011109,
25011120,
25011119,
25011011,
25010940,
25011010,
25011009,
25010933,
25010857,
25010817,
25010816,
25010815,
25010696,
25010692,
25010663,
25010664,
25010665,
25010596,
25012963,
25012973,
25012986,
25012688,
25012758,
25012982,
25012981,
25012980,
25010801,
25010770,
25010966,
25010990,
25011340,
25011339,
25011279,
25011221,
25011223,
25011417,
25011166,
25011068,
25011611,
25011551,
25011583,
25011939,
25011905,
25012004,
25012505,
25012360,
25012359,
25012556,
25012336,
25012174,
25012933,
25019628,
25019574,
25020053,
25021358,
25021365,
25021349,
25023631,
25023586,
25023392,
25023061,
25023105,
25023102,
25022977,
25022947,
25022890,
25022731,
25022858,
25022736,
25022613,
25022131,
25022125,
25022266,
25022410,
25021970,
25021881,
25019363,
25021705,
25021551,
25021358,
25021365,
25021349,
25015431,
25014767,
25014766,
25014765,
25014614,
25014613,
25014612,
25014925,
25014200,
25014461,
25014460,
25014315,
25013872,
25013971,
25013258,
25013257,
25014548,
25014791,
25014857,
25014941,
25013897,
25013877,
25013663,
25013668,
25013660,
25013541,
25013615,
25015383,
25015361,
25015353,
25015354,
25015351,
25015165,
25015187,
25015180,
25015177,
25015237,
25015005,
25015050,
25014993,
25014774,
25014660,
25014661,
25014662,
25014472,
25014263,
25013929,
25014050,
25013661,
25013672,
25013646,
25013483,
25013388,
25013335,
25013101,
25015549,
25019124,
25018633,
25018632,
25018661,
25019569,
25019469,
25019470,
25019471,
25020686,
25020685
]

In [5]:
us_ig = us_ig.loc[(~us_ig['DealId'].isin(exclude_these))]

In [6]:
#change date to dtype datetime64[ns]
us_ig['PricingDate']=pd.to_datetime(us_ig['PricingDate'])

In [7]:
#rename columns
us_ig.rename(columns={'Size (m)': 'Size_m', 'Book Size (m)': 'Book_Size'}, inplace=True)

In [8]:
#compute bk_to_cvr
us_ig['tranche_bk_to_cvr'] = round(us_ig['Book_Size']/us_ig['Size_m'], 2)

In [9]:
#Extract quarter, year, week, month from dates
us_ig['quarter'] = pd.PeriodIndex(us_ig.PricingDate, freq = 'Q')
us_ig['month']=pd.PeriodIndex(us_ig.PricingDate, freq = 'm')
us_ig['week']=us_ig.PricingDate.dt.strftime('%Y-%W')
us_ig['year']=pd.PeriodIndex(us_ig.PricingDate, freq = 'y')

In [11]:
#normalizing tenor 
us_ig['normalized_tenor'] = us_ig['Tenor'].fillna(0.0).astype(int)

In [13]:
#saving to csv all historical ig
us_ig.to_csv('../Dash_app/data/us_ig_cleaned.csv')

In [14]:
ig_ytd = us_ig.loc[us_ig['PricingDate']>='2022-01-01']

In [15]:
ig_ytd['Nic'] = pd.to_numeric(ig_ytd['Nic'], errors='coerce')

In [16]:
tripple_a = ['Aaa','AAA']
double_a = ['Aa1','AA+','Aa2','AA','Aa3','AA-']
single_a = ['A1','A2','A3','A+','A','A-']
tripple_b =['Baa1','Baa2','Baa3','BBB+','BBB','BBB-']

In [17]:
conditions = [
    (ig_ytd['S&P'].isin(tripple_a)) | (ig_ytd['Moodys'].isin(tripple_a)),
    (ig_ytd['S&P'].isin(double_a)) | (ig_ytd['Moodys'].isin(double_a)),
    (ig_ytd['S&P'].isin(single_a)) | (ig_ytd['Moodys'].isin(single_a)),
    (ig_ytd['S&P'].isin(tripple_b)) | (ig_ytd['Moodys'].isin(tripple_b))
]

values = ['AAA', 'AA', 'A', 'BBB']

ig_ytd['ratings'] = np.select(conditions, values)

In [18]:
#saving to csv ig_ytd
ig_ytd.to_csv('../Dash_app/data/ig_ytd.csv')